In [1]:
from google.colab import drive
drive.mount('/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


**첫번째 구현하는 모델은 패킷의 모든 필드를 사용하여 학습한다**

In [4]:
#필요 없는 열은 제거하기
import pandas as pd
import numpy as np

data = pd.read_csv("/gdrive/My Drive/classification/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df_sample = data.loc[:, [' Source Port',' Destination Port', ' Protocol', ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size', ' Fwd Header Length', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward', ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min',' Label']]
df_labels= df_sample[' Label']
df_packets=df_sample.drop([' Label'],axis='columns')

#같은 길이 나오는 거확인
print(len(df_sample))
print(len(df_labels))
print(len(df_packets))


#콜롬 목록과 데이터 타입 확인 후 변환
# df_labels.columns #series라 안됨
df_packets.columns
#type(df_sample)
df_packets = df_packets.astype(float) #형변환

#리스트로 되돌리기
packets= df_packets.values.tolist()
labels= df_labels.values.tolist()


#확인
packets[1]
labels[1]


#길이 확인
len(packets)
len(labels)
#len(packets[1]) #한개당 80차원

# labels도 float으로 바꿔주기
for c in range(225745):
  if (labels[c] == 'BENIGN' ):
    labels[c]=0 #false
  else:
    labels[c] =1 #true
 

#테스트 군과 학습군으로 나누기.
#전체는 225745개. (70%는 약 158,021 개)
train_set = packets[:158021]
test_set = packets[158021:]
train_label =labels[:158021]
test_label = labels[158021:]

train_set = np.asarray(train_set)
train_label = np.asarray(train_label)
test_set = np.asarray(test_set)
test_label = np.asarray(test_label)


#잘 나눠졌는지 확인
print(len(train_set) + len(test_set))
print(len(test_label) + len(train_label))

#모델만들기
import numpy as np

import tensorflow as tf

model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, (2), activation='relu', input_shape=(80, 1)),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Conv1D(16, (2), activation='relu'),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(15, activation='softmax')
])
model.summary() #모델 출력


import numpy as np

train_set = np.expand_dims(train_set, axis=2) #reshape
test_set = np.expand_dims(test_set, axis=2) #reshape


#학습.
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_set, train_label, epochs=10, #10번의 epoch
                    validation_data=(test_set, test_label))
test_loss = model.evaluate(test_set, test_label)





225745
225745
225745
225745
225745
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 79, 16)            48        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 39, 16)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 16)            528       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 19, 16)            0         
_________________________________________________________________
flatten (Flatten)            (None, 304)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               156160    
_________________________________________________________________
dense_1 (Dense)      